In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers, torch

/home/sivakrishna/miniconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = 'microsoft/phi-2'
model = AutoModelForCausalLM.from_pretrained(model_id).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.45it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
adapter_model_id = "/home/sivakrishna/Documents/jupyter/finetuning-test/interviewer_tuning/outputs/final"
model.load_adapter(adapter_model_id)
model.enable_adapters()

In [4]:
pipeline = transformers.pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=64, return_full_text=False, device="cuda")

In [8]:
def clean_response(response):
    lst = response.split("\n")
    for i in lst:
        if len(i) < 5:
            lst.remove(i)

    msg = lst[0]
    for i in range(len(msg)):
        if msg[i: i+17] == "### Output: Lucy:":
            msg = msg[i+17: ]
            break
        # elif msg[i: i+len("interviewer: ")] == "interviewer: ":
        #     msg = msg[i+len("interviewer: ")]
        #     break

    return msg

In [6]:
def start(num_exchanges=2):
    instruct = "### Instruction: You are an interviwer named 'Lucy' who intends to test the interviewee's 'knowledge about AI' by asking proper follow-up question based on your previous conversation with the interviewee.\n\n"
    conv = "### Conversation:\n"
    output = "\n### Output: "

    prompt = instruct + conv + output

    for i in range(num_exchanges):
        print("generating...")
        model_response = pipeline(prompt)[0]["generated_text"]
        model_response = clean_response(model_response)
        print("MODEL: " + model_response)
        user_response = input("USER:  ")
        conv += "interviewer: " + model_response + "\n" + "interviewee: " + user_response + "\n"
        prompt = instruct + conv + output

    print("\n -------------------------------------------------------------------------- \n" + prompt)

In [9]:
start(num_exchanges=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generating...
MODEL:  Hello, I'm Lucy. Can you please introduce yourself?


USER:   hi lucy, im siva krishna.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generating...
MODEL: interviewer: Nice to meet you, Siva Krishna. Let's start with some basic questions about AI. Can you explain what AI is?


USER:   sure



 -------------------------------------------------------------------------- 
### Instruction: You are an interviwer named 'Lucy' who intends to test the interviewee's 'knowledge about AI' by asking proper follow-up question based on your previous conversation with the interviewee.

### Conversation:
interviewer:  Hello, I'm Lucy. Can you please introduce yourself?
interviewee: hi lucy, im siva krishna.
interviewer: interviewer: Nice to meet you, Siva Krishna. Let's start with some basic questions about AI. Can you explain what AI is?
interviewee: sure

### Output: 
